In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output as ipython_clear_output

# Import the refactored package
import nrw_geotools as ngt
# For direct access to callback functions, if not re-exported via ngt.__init__
from nrw_geotools import callbacks 

# 1. Initialize Application Context
# This dictionary will hold all shared components and be passed to callbacks.
app_context = {
    "m": None,
    "widgets": None,
    "config": ngt.config,
    "state": ngt.state, # Direct access to the state module
}

# 2. Create Map
app_context["m"] = ngt.map_setup.create_map()
ngt.map_setup.add_initial_layers(app_context["m"])

# 3. Create UI Widgets
app_context["widgets"] = ngt.ui_manager.create_widgets()

# File operations
app_context["widgets"]['save_selected_as_gml_button'].on_click(
    lambda b: callbacks.on_save_gml_button_clicked(app_context)
)

# 4. Bind Callbacks
#    We use lambdas to pass the app_context to our callback handlers.

# WFS related
app_context["widgets"]['discover_button'].on_click(
    lambda b: callbacks.on_discover_button_clicked(app_context)
)
app_context["widgets"]['fetch_data_button'].on_click(
    lambda b: callbacks.on_fetch_data_button_clicked(app_context)
)

# Map's main draw control actions (for user-drawn features)
# The handler in feature_manager expects (draw_control_instance, action, geo_json, app_context)
app_context["m"].draw_control.on_draw(
    lambda dc_instance, action, geo_json: callbacks.on_draw_control_action(
        dc_instance, action, geo_json, app_context
    )
)

# Feature management buttons
app_context["widgets"]['keep_selected_button'].on_click(
    lambda b: callbacks.on_keep_selected_button_clicked(app_context)
)
app_context["widgets"]['clear_selection_button'].on_click(
    lambda b: callbacks.on_clear_selection_button_clicked(app_context)
)
app_context["widgets"]['remove_selected_button'].on_click(
    lambda b: callbacks.on_remove_selected_button_clicked(app_context)
)

# Feature editing buttons
app_context["widgets"]['edit_selected_feature_button'].on_click(
    lambda b: callbacks.on_edit_selected_feature_button_clicked(app_context)
)
app_context["widgets"]['apply_feature_edits_button'].on_click(
    lambda b: callbacks.on_apply_feature_edits_button_clicked(app_context)
)
app_context["widgets"]['cancel_feature_edits_button'].on_click(
    lambda b: callbacks.on_cancel_feature_edits_button_clicked(app_context)
)

# Feature cutting buttons
app_context["widgets"]['cut_selected_button'].on_click(
    lambda b: callbacks.on_cut_selected_button_clicked(app_context)
)
app_context["widgets"]['cancel_cut_button'].on_click(
    lambda b: callbacks.on_cancel_cut_button_clicked(app_context)
)

# File operations
app_context["widgets"]['save_selected_as_gml_button'].on_click(
    lambda b: callbacks.on_save_gml_button_clicked(app_context)
)


# 5. Initialize Drawn Features Layer (after map and basic callbacks are set up)
# The on_click_callback for the drawn_features_layer needs to be a lambda
# that calls the generic on_geojson_feature_click_callback_base with the correct layer name and app_context.
drawn_features_on_click_lambda = callbacks.get_geojson_click_handler(
    app_context, app_context["config"].DRAWN_FEATURES_LAYER_NAME
)
ngt.map_setup.initialize_drawn_features_layer_on_map(
    app_context["m"],
    drawn_features_on_click_lambda
)
# The layer object is stored in app_context["state"].drawn_features_layer by the function above.


# 6. Layout UI
ui_layout = ngt.ui_manager.layout_widgets(app_context["widgets"])

# 7. Display UI and Map
display(ui_layout)
display(app_context["m"])

# 8. Initial Button State Update
ngt.ui_manager.update_all_button_states(app_context)

print("Application setup complete.")